In [ ]:
import os
import ase
from ase import Atoms
import numpy as np
import tqdm
import ase.io
from nice.blocks import *
from nice.utilities import *
from matplotlib import pyplot as plt
from sklearn.linear_model import BayesianRidge

In [ ]:
PROPERTIES_NAMES = [
    'tag', 'index', 'A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2',
    'zpve', 'U0', 'U', 'H', 'G', 'Cv'
]


def string_to_float(element):
    '''because shit like 2.1997*^-6 happens'''
    return float(element.replace('*^', 'e'))


PROPERTIES_HANDLERS = [str, int
                       ] + [string_to_float] * (len(PROPERTIES_NAMES) - 2)


def parse_qm9_xyz(path):
    with open(path, 'r') as f:
        lines = list(f)
    #print(lines)
    n_atoms = int(lines[0])
    properties = {
        name: handler(value)
        for handler, name, value in zip(PROPERTIES_HANDLERS, PROPERTIES_NAMES,
                                        lines[1].strip().split())
    }
    composition = ""
    positions = []
    for i in range(2, 2 + n_atoms):
        composition += lines[i].strip().split()[0]
        positions.append([
            string_to_float(value) for value in lines[i].strip().split()[1:4]
        ])

    positions = np.array(positions)
    result = Atoms(composition, positions=np.array(positions))
    result.info.update(properties)
    return result


def parse_index(path):
    with open(path, "r") as f:
        lines = list(f)
    proper_lines = lines[9:-1]
    result = [int(line.strip().split()[0]) for line in proper_lines]
    return np.array(result, dtype=int)


def download_qm9(clean=True):
    #downloading from https://figshare.com/collections/Quantum_chemistry_structures_and_properties_of_134_kilo_molecules/978904
    os.system(
        "wget https://ndownloader.figshare.com/files/3195389 -O qm9_main.xyz.tar.bz2"
    )
    os.system(
        "wget https://ndownloader.figshare.com/files/3195404 -O problematic_index.txt"
    )
    os.system("mkdir qm9_main_structures")
    os.system("tar xjf qm9_main.xyz.tar.bz2 -C qm9_main_structures")

    names = [
        name for name in os.listdir('qm9_main_structures/')
        if name.endswith('.xyz')
    ]
    names = sorted(names)

    structures = [
        parse_qm9_xyz('qm9_main_structures/{}'.format(name))
        for name in tqdm.tqdm(names)
    ]

    problematic_index = parse_index('problematic_index.txt')
    np.save('problematic_index.npy', problematic_index)
    ase.io.write('qm9_main.extxyz', structures)
    if (clean):
        os.system("rm -r qm9_main_structures")
        os.system("rm problematic_index.txt")
        os.system("rm qm9_main.xyz.tar.bz2")
    return structures, problematic_index


def get_qm9(clean=True):
    if ('qm9_main.extxyz' in os.listdir('.')) and \
              ('problematic_index.npy' in os.listdir('.')):
        structures = ase.io.read('qm9_main.extxyz', index=':')
        problematic_index = np.load('problematic_index.npy')
        return structures, problematic_index
    else:
        return download_qm9(clean=clean)

In [ ]:
structures, problematic_index = get_qm9()

In [ ]:
HARTREE_TO_EV = 27.211386245988
USE_PROBLEMATIC_INDEX = False
np.random.seed(0)

if (not USE_PROBLEMATIC_INDEX):
    structures = [
        structure for structure in structures
        if structure.info['index'] not in problematic_index
    ]

del problematic_index  #it borrows indexing from 1 from qm9, deleting it away from sin

permutation = np.random.permutation(len(structures))
train_indices = permutation[0:2000]
test_indices = permutation[2000:2500]
environments_for_fitting = 1000  #number of environments to fit nice transfomers
grid = [150, 200, 350, 500, 750, 1000, 1500, 2000]  #for learning curve

#HYPERS for librascal spherical expansion coefficients
HYPERS = {
    'interaction_cutoff': 5,
    'max_radial': 15,
    'max_angular': 5,
    'gaussian_sigma_type': 'Constant',
    'gaussian_sigma_constant': 0.05,
    'cutoff_smooth_width': 0.3,
    'radial_basis': 'GTO'
}

In [ ]:
#our model:
def get_nice():
    return StandardSequence([
        StandardBlock(ThresholdExpansioner(num_expand=300),
                      CovariantsPurifierBoth(max_take=10),
                      IndividualLambdaPCAsBoth(n_components=100),
                      ThresholdExpansioner(num_expand=1000, mode='invariants'),
                      InvariantsPurifier(max_take=10),
                      InvariantsPCA(n_components=200)),
        StandardBlock(ThresholdExpansioner(num_expand=300),
                      CovariantsPurifierBoth(max_take=10),
                      IndividualLambdaPCAsBoth(n_components=100),
                      ThresholdExpansioner(num_expand=1000, mode='invariants'),
                      InvariantsPurifier(max_take=10),
                      InvariantsPCA(n_components=200)),
        StandardBlock(None, None, None,
                      ThresholdExpansioner(num_expand=1000, mode='invariants'),
                      InvariantsPurifier(max_take=10),
                      InvariantsPCA(n_components=100))
    ],
                            initial_scaler=InitialScaler(
                                mode='signal integral', individually=True))

In [ ]:
train_structures = [structures[i] for i in train_indices]
test_structures = [structures[i] for i in test_indices]

all_species = get_all_species(train_structures + test_structures)

train_coefficients = get_spherical_expansion(train_structures, HYPERS,
                                             all_species)
test_coefficients = get_spherical_expansion(test_structures, HYPERS,
                                            all_species)

In [ ]:
all_coefficients = [
    train_coefficients[key] for key in train_coefficients.keys()
]
all_coefficients = np.concatenate(all_coefficients, axis=0)
np.random.shuffle(all_coefficients)
all_coefficients = all_coefficients[0:environments_for_fitting]

In [ ]:
nice_single = get_nice()
nice_single.fit(all_coefficients)

In [ ]:
# using same nice transformer regardless of central specie
nice = {specie: nice_single for specie in all_species}

In [ ]:
train_features = transform_sequentially(nice, train_structures, HYPERS,
                                        all_species)
test_features = transform_sequentially(nice, test_structures, HYPERS,
                                       all_species)

In [ ]:
train_c_features = get_compositional_features(train_structures, all_species)
test_c_features = get_compositional_features(test_structures, all_species)

train_features = np.concatenate([train_features, train_c_features], axis=1)
test_features = np.concatenate([test_features, test_c_features], axis=1)

In [ ]:
train_energies = [structure.info['U0'] for structure in train_structures]
train_energies = np.array(train_energies) * HARTREE_TO_EV

test_energies = [structure.info['U0'] for structure in test_structures]
test_energies = np.array(test_energies) * HARTREE_TO_EV

In [ ]:
def get_rmse(first, second):
    return np.sqrt(np.mean((first - second)**2))


def get_mae(first, second):
    return np.mean(np.abs(first - second))


def estimate_performance(regressor, data_train, data_test, targets_train,
                         targets_test):
    regressor.fit(data_train, targets_train)
    predictions = regressor.predict(data_test)
    return get_rmse(predictions,
                    targets_test), get_mae(predictions, targets_test)

In [ ]:
errors_compositional = []
for el in tqdm.tqdm(grid):
    errors_compositional.append(
        estimate_performance(BayesianRidge(), train_c_features[:el],
                             test_c_features, train_energies[:el],
                             test_energies))

errors_compositional = np.array(errors_compositional)
errors_nice = []
for el in tqdm.tqdm(grid):
    # because without this step with residuals
    # joint fitting might face problems due to
    # regularization
    regressor = BayesianRidge()
    regressor.fit(train_c_features[:el], train_energies[:el])

    residuals_train = train_energies[:el] - regressor.predict(
        train_c_features[:el])
    residuals_test = test_energies - regressor.predict(test_c_features)

    errors_nice.append(
        estimate_performance(BayesianRidge(), train_features[:el],
                             test_features, residuals_train, residuals_test))

errors_nice = np.array(errors_nice)

In [ ]:
fig, axes = plt.subplots(1, 2)

axes[0].plot(grid, errors_compositional[:, 0], 'ro')
axes[0].plot(grid, errors_compositional[:, 0], 'r', label='only compositional')

axes[0].plot(grid, errors_nice[:, 0], 'bo')
axes[0].plot(grid, errors_nice[:, 0], 'b', label='nice')

axes[0].set_xlabel("n_train")
axes[0].set_ylabel("rmse, eV")
axes[0].set_xscale('log')
axes[0].set_yscale('log')

axes[1].plot(grid, errors_compositional[:, 1], 'ro')
axes[1].plot(grid, errors_compositional[:, 1], 'r', label='only compositional')

axes[1].plot(grid, errors_nice[:, 1], 'bo')
axes[1].plot(grid, errors_nice[:, 1], 'b', label='nice')

axes[1].set_xlabel("n_train")
axes[1].set_ylabel("mae, eV")
axes[1].set_xscale('log')
axes[1].set_yscale('log')
plt.legend(loc='upper center')
plt.subplots_adjust(wspace=0.4)
plt.show()

In [ ]:
print("mae: {} eV".format(errors_nice[-1][0]))
print("rmse: {} eV".format(errors_nice[-1][1]))